# Timeseries Forecasting (NHITS model)

## 1. Installing dependencies

In [ ]:
# pip install neuralforecast 

In [ ]:
# pip install sktime

In [ ]:
# pip install datasetsforecast

In [ ]:
# Basics
import pandas as pd
import numpy as np

# Some functions for plotting and stuff
import utils as ts_utils

# Neural models
from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS
from neuralforecast.losses.pytorch import MQLoss

## 2. Data Preparation

In [ ]:
# Size of the data to read
data_size = 'norm'

# Date of the data to read
data_date = '2110' # '1806' = 18th of June

# Read the data (takes around 2 minutes)
dataset = pd.read_csv(f"~/Thesis/data/eod_balances_{data_date}_{data_size}.csv")

dataset

In [ ]:
# Create the timer
timer = ts_utils.Timer()

### 2.1 In-sample and Out-sample split

In [ ]:
# Calculate total amount of timeseries
num_timeseries = len(dataset.columns) - 1

# Specify train test split percentage
train_test_split = 0.8

# Split into train and out of sample test data
num_out_of_sample = int(train_test_split * num_timeseries)

# Create in-sample dataframe
in_sample_data = dataset.iloc[:, : num_out_of_sample + 1] # Training and testing

# Create out-sample dataframe
n = num_timeseries-num_out_of_sample
columns_to_keep = dataset.columns[[0]].tolist() + dataset.columns[-n:].tolist()
out_sample_data = dataset[columns_to_keep]

## 3. In-sample Analysis

### 3.1 Train/Test splitting and plotting

In [ ]:
# Change the data to the long format
Y_df = in_sample_data.melt(id_vars=['date'], var_name='unique_id', value_name='y')
Y_df = Y_df.rename(columns={'date':'ds'})

# Convert date column to datetime type
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

In [ ]:
# Define the horizon (12 months of 30 days each)
fh = 30
horizon = 12 * fh

# Identify the unique dates in the dataset
unique_dates = Y_df['ds'].unique()

# Convert to a list and then sort the dates
unique_dates = sorted(list(unique_dates))

# Determine the cutoff date (cutoff at 12 months before the last date in the dataset)
cutoff_date = unique_dates[-(horizon + 1)]

# Training data: all data up to the cutoff date
Y_train_df = Y_df[Y_df['ds'] <= cutoff_date]

In [ ]:
# Initialize lists to store the input and test sets
input_dfs = []
test_dfs = []

# Loop to create the 6 input and test sets
for i in range(6):
    # Determine the start date of the test period
    test_start_date = unique_dates[-(horizon - i * 2 * fh)]
    test_end_date = unique_dates[-(horizon - (i * 2 * fh) - fh)]
    
    # Input data: all data up to the start of the current test period
    input_df = Y_df[Y_df['ds'] <= test_start_date]
    input_dfs.append(input_df)
    
    # Test data: the 30-day period following the start of the test period
    test_df = Y_df[(Y_df['ds'] > test_start_date) & (Y_df['ds'] <= test_end_date)]
    test_dfs.append(test_df)

# Define the 6 input periods
Y_input_df_0 = input_dfs[0]
Y_input_df_1 = input_dfs[1]
Y_input_df_2 = input_dfs[2]
Y_input_df_3 = input_dfs[3]
Y_input_df_4 = input_dfs[4]
Y_input_df_5 = input_dfs[5]

# Define the 6 test periods
Y_test_df_0 = test_dfs[0]
Y_test_df_1 = test_dfs[1]
Y_test_df_2 = test_dfs[2]
Y_test_df_3 = test_dfs[3]
Y_test_df_4 = test_dfs[4]
Y_test_df_5 = test_dfs[5]

In [ ]:
# Timeserie to plot
unique_id = '6'

# Plot the train and test dataframes
ts_utils.plot_train_test_split(Y_input_df_0, Y_test_df_0, unique_id)

### 3.2 Training and predicting

In [ ]:
# Decide max_steps:
if data_size == 'tiny':
    max_steps = 500   #checked 
if data_size == 'full':
    max_steps = 3500 # 3500 is checked == 500 epochs
if data_size == 'norm':
    max_steps = 3500

print(f"The maximum steps we take during training will be {max_steps}.")

#### 3.2.1 Train the model

In [ ]:
# Set forecasting horizon
fh = 30

# Set our confidence levels
levels = [60, 70, 80, 90]

timer.record_timestamp("start_train")

# Initialize the model
nhits_model_insample = NeuralForecast(models=[NHITS(h=fh,
                                                    input_size= 15 * fh,
                                                    loss=MQLoss(level=levels),
                                                    scaler_type='standard',
                                                    max_steps=max_steps,
                                                    val_check_steps=max_steps / 10,
                                                    n_freq_downsample=[2, 1, 1])
                                             ], freq='D'
                                     )

# Fit the model
nhits_model_insample.fit(df=Y_train_df)

timer.record_timestamp("end_train")

#### 3.2.1 Retrieve predictions

In [ ]:
timer.record_timestamp("start_inference")

# Make predictions on first period
nhits_model_insample_preds_0 = nhits_model_insample.predict(df=Y_input_df_0).rename(columns={'NHITS-median': 'NHITS'})

timer.record_timestamp("end_inference")

# Save predictions of first period
nhits_model_insample_preds_0.to_csv(f'~/Thesis/predictions/NHITS/insample/period01/model_preds_{data_date}_{data_size}.csv', index=True)

In [ ]:
# Make predictions on second period
nhits_model_insample_preds_1 = nhits_model_insample.predict(df=Y_input_df_1).rename(columns={'NHITS-median': 'NHITS'})

# Save predictions of second period
nhits_model_insample_preds_1.to_csv(f'~/Thesis/predictions/NHITS/insample/period02/model_preds_{data_date}_{data_size}.csv', index=True)

In [ ]:
# Make predictions on third period
nhits_model_insample_preds_2 = nhits_model_insample.predict(df=Y_input_df_2).rename(columns={'NHITS-median': 'NHITS'})

# Save predictions of third period
nhits_model_insample_preds_2.to_csv(f'~/Thesis/predictions/NHITS/insample/period03/model_preds_{data_date}_{data_size}.csv', index=True)

In [ ]:
# Make predictions on fourth period
nhits_model_insample_preds_3 = nhits_model_insample.predict(df=Y_input_df_3).rename(columns={'NHITS-median': 'NHITS'})

# Save predictions of fourth period
nhits_model_insample_preds_3.to_csv(f'~/Thesis/predictions/NHITS/insample/period04/model_preds_{data_date}_{data_size}.csv', index=True)

In [ ]:
# Make predictions on fifth period
nhits_model_insample_preds_4 = nhits_model_insample.predict(df=Y_input_df_4).rename(columns={'NHITS-median': 'NHITS'})

# Save predictions of fifth period
nhits_model_insample_preds_4.to_csv(f'~/Thesis/predictions/NHITS/insample/period05/model_preds_{data_date}_{data_size}.csv', index=True)

In [ ]:
# Make predictions on sixth period
nhits_model_insample_preds_5 = nhits_model_insample.predict(df=Y_input_df_5).rename(columns={'NHITS-median': 'NHITS'})

# Save predictions of sixth period
nhits_model_insample_preds_5.to_csv(f'~/Thesis/predictions/NHITS/insample/period06/model_preds_{data_date}_{data_size}.csv', index=True)

## 4. Out-of-sample Analysis

### 4.1 Data Handling

In [ ]:
# Change the data to the long format
Y_df = out_sample_data.melt(id_vars=['date'], var_name='unique_id', value_name='y')
Y_df = Y_df.rename(columns={'date':'ds'})

# Convert date column to datetime type
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

In [ ]:
# Define the horizon (12 months of 30 days each)
fh = 30
horizon = 12 * fh

# Identify the unique dates in the dataset
unique_dates = Y_df['ds'].unique()

# Convert to a list and then sort the dates
unique_dates = sorted(list(unique_dates))

# Determine the cutoff date (cutoff at 12 months before the last date in the dataset)
cutoff_date = unique_dates[-(horizon + 1)]

# Training data: all data up to the cutoff date
Y_train_df = Y_df[Y_df['ds'] <= cutoff_date]

In [ ]:
# Initialize lists to store the input and test sets
input_dfs = []
test_dfs = []

# Loop to create the 6 input and test sets
for i in range(6):
    # Determine the start date of the test period
    test_start_date = unique_dates[-(horizon - i * 2 * fh)]
    test_end_date = unique_dates[-(horizon - (i * 2 * fh) - fh)]
    
    # Input data: all data up to the start of the current test period
    input_df = Y_df[Y_df['ds'] <= test_start_date]
    input_dfs.append(input_df)
    
    # Test data: the 30-day period following the start of the test period
    test_df = Y_df[(Y_df['ds'] > test_start_date) & (Y_df['ds'] <= test_end_date)]
    test_dfs.append(test_df)

# Define the 6 input periods
Y_input_df_0 = input_dfs[0]
Y_input_df_1 = input_dfs[1]
Y_input_df_2 = input_dfs[2]
Y_input_df_3 = input_dfs[3]
Y_input_df_4 = input_dfs[4]
Y_input_df_5 = input_dfs[5]

# Define the 6 test periods
Y_test_df_0 = test_dfs[0]
Y_test_df_1 = test_dfs[1]
Y_test_df_2 = test_dfs[2]
Y_test_df_3 = test_dfs[3]
Y_test_df_4 = test_dfs[4]
Y_test_df_5 = test_dfs[5]

In [ ]:
# Timeserie to plot
unique_id = Y_train_df['unique_id'][0]

# Plot the train and test dataframes
ts_utils.plot_train_test_split(Y_train_df, Y_test_df_0, unique_id)

### 4.2 Retrieve predictions

In [ ]:
# Make outsample predictions on first period (using already trained model)
nhits_model_outsample_preds_0 = nhits_model_insample.predict(df=Y_input_df_0).rename(columns={'NHITS-median': 'NHITS'})

# Save outsample predictions of first period
nhits_model_outsample_preds_0.to_csv(f'~/Thesis/predictions/NHITS/outsample/period01/model_preds_{data_date}_{data_size}.csv', index=True)

In [ ]:
# Make outsample predictions on second period (using already trained model)
nhits_model_outsample_preds_1 = nhits_model_insample.predict(df=Y_input_df_1).rename(columns={'NHITS-median': 'NHITS'})

# Save outsample predictions of second period
nhits_model_outsample_preds_1.to_csv(f'~/Thesis/predictions/NHITS/outsample/period02/model_preds_{data_date}_{data_size}.csv', index=True)

In [ ]:
# Make outsample predictions on third period (using already trained model)
nhits_model_outsample_preds_2 = nhits_model_insample.predict(df=Y_input_df_2).rename(columns={'NHITS-median': 'NHITS'})

# Save outsample predictions of third period
nhits_model_outsample_preds_2.to_csv(f'~/Thesis/predictions/NHITS/outsample/period03/model_preds_{data_date}_{data_size}.csv', index=True)

In [ ]:
# Make outsample predictions on fourth period (using already trained model)
nhits_model_outsample_preds_3 = nhits_model_insample.predict(df=Y_input_df_3).rename(columns={'NHITS-median': 'NHITS'})

# Save outsample predictions of fourth period
nhits_model_outsample_preds_3.to_csv(f'~/Thesis/predictions/NHITS/outsample/period04/model_preds_{data_date}_{data_size}.csv', index=True)

In [ ]:
# Make outsample predictions on fifth period (using already trained model)
nhits_model_outsample_preds_4 = nhits_model_insample.predict(df=Y_input_df_4).rename(columns={'NHITS-median': 'NHITS'})

# Save outsample predictions of fifth period
nhits_model_outsample_preds_4.to_csv(f'~/Thesis/predictions/NHITS/outsample/period05/model_preds_{data_date}_{data_size}.csv', index=True)

In [ ]:
# Make outsample predictions on sixth period (using already trained model)
nhits_model_outsample_preds_5 = nhits_model_insample.predict(df=Y_input_df_5).rename(columns={'NHITS-median': 'NHITS'})

# Save outsample predictions of fifth period
nhits_model_outsample_preds_5.to_csv(f'~/Thesis/predictions/NHITS/outsample/period06/model_preds_{data_date}_{data_size}.csv', index=True)

### Model Statistics

In [ ]:
train_time = timer.elapsed_time("start_train", "end_train")
inference_time = timer.elapsed_time("start_inference", "end_inference")

model_name = "NHITS"
num_timeseries = 277

file_path = "model_statistics.txt"

ts_utils.write_statistics(model_name, num_timeseries, train_time, inference_time, file_path)